In [ ]:
# general
import json
import collections
import math
import uuid
import random
import pickle
import os
import time
from itertools import islice
import numpy as np
from tqdm import tqdm_notebook as tqdm

from src.geometry import sq_distance, vector_mag, sq_dist_vect
from src.elements import *

In [ ]:
blueprint = 'data/sample.ifc'
data_path = "/mnt/c/data/3D_CAD/"
dist_threshold = 0.001 # distance threshold for pipe proximity
centerline_angle_threshold = math.radians(10) # angle threshold for two pipes to be considered parallel
centerline_dist_threshold = 0.0005 # centerline distance threshold pipes to be to considered connected

In [ ]:
# load data
with open(data_path + 'edges_westdeckbox.pkl', 'rb') as f:
    edges = pickle.load(f)
with open(data_path + 'nodes_westdeckbox.pkl', 'rb') as f:
    node_info = pickle.load(f)
    nodes = node_info[0]
    


In [ ]:
print(nodes[0], edges[0])

In [ ]:
e1 = [e[0] for e in edges]
e2 = [e[1] for e in edges]
print(max(e1), max(e2))
print(len(nodes))
print(len(edges))

In [ ]:
print(type(nodes[0][3]))

In [ ]:
# find closest pair of edges satisfying a minimum distance criteria
def edge_proximity_criteria(e1, e2, threshold):
    nearby_pair = None
    min_dist = math.inf
    d = sq_dist_vect(e1[0], e2[0])
    if (d < threshold):
        nearby_pair = [0, 0]
        min_dist = d
    d = sq_dist_vect(e1[1], e2[0])
    if (d < threshold and d < min_dist): 
        nearby_pair = [1, 0]
        min_dist = d
    d = sq_dist_vect(e1[0], e2[1])
    if (d < threshold and d < min_dist): 
        nearby_pair = [0, 1]
        min_dist = d    
    d = sq_dist_vect(e1[1], e2[1]) 
    if (d < threshold and d < min_dist): 
        nearby_pair = [1, 1]
        min_dist = d
    return nearby_pair


In [ ]:
# get pipe neighbours

# find edges of pipes
# format - category, centerpoint, bounding box, principal direction, element id
pipes = [[n[0], np.array(n[1]), np.array(n[2]), np.array(n[3]), n[4]] for n in nodes if n[0]==3]
pipe_edges = [(n[1] - (max(n[2])/2 * n[3]),
               n[1] + (max(n[2])/2 * n[3])) for n in pipes]
# print(len(pipe_edges))
# print(pipe_edges[0])

# compare edges to find close-by edges
nearby_edges = []
for i, pe1 in tqdm(enumerate(pipe_edges)):
    t1 = time.perf_counter()
    for j, pe2 in enumerate(pipe_edges[i+1:]):
        nearby_pair = edge_proximity_criteria(pe1, pe2, dist_threshold)
        if nearby_pair is not None:
            nearby_edges.append(((i,j+i+1), nearby_pair))
    t2 = time.perf_counter()
    if (t2 - t2) > 0.1:
        print((t2 - t1), nearby_pair, min_dist)            
print(len(nearby_edges), nearby_edges[0])

In [ ]:
print(pipe_edges[0])
print(pipes[10])

In [ ]:
nearby_edges[:10]

In [ ]:
def edge_match(a, b):
    e = None
    if (a[0][0] == b[0][0] and a[1][0] == b[1][0]):
        e = 1
    elif (a[0][1] == b[0][0] and a[1][1] == b[1][0]):
        e = 0
    elif (a[0][0] == b[0][1] and a[1][0] == b[1][1]):
        e = 1
    elif (a[0][1] == b[0][1] and a[1][1] == b[1][1]):
        e = 0
    
    if e is not None:
        return ((a[0][e], b[0][0], b[0][1]), (a[1][e], b[1][0], b[1][1]))
    else:
        return None

In [ ]:
# rough check for tees
possible_tees = []
for i, ne1 in tqdm(enumerate(nearby_edges)):
    for j, ne2 in enumerate(nearby_edges[i+1:]):
        pt = edge_match(ne1, ne2)
        if pt is not None:
            possible_tees.append(pt)
            
print(len(possible_tees), possible_tees[0])

In [ ]:
def pipe_check(a, b):
    # centerline direction check -
    centerline_deviation = np.arccos( np.dot(a[3], b[3]))
    if centerline_deviation > np.pi/2:
        centerline_deviation = np.pi - centerline_deviation
    if centerline_deviation < centerline_angle_threshold:
        return True
    
def elbow_check(a, b):
    # centerline proximity check
    centerline_connecting_line = np.cross(a[3], b[3])
    edge_connecting_line = b[1] - a[1]
    centerline_distance = (abs(np.dot(centerline_connecting_line, 
                                          edge_connecting_line)) / 
                           vector_mag(centerline_connecting_line))

    if centerline_distance < centerline_dist_threshold:
        return True

In [ ]:
# check for tees
# if two of the elements seem to be connected in a straight line and the other is placed like an elbow, its considered a tee
tee_connections = []
for i, pt in tqdm(enumerate(possible_tees)):
    pipe_connection = False
    if pipe_check(pipes[pt[0][0]], pipes[pt[0][1]]):
        pipe_connection = True
        pipe_pair = [0,1]
        other = 2
    if pipe_check(pipes[pt[0][0]], pipes[pt[0][2]]):
        pipe_connection = True
        pipe_pair = [0,2]
        other = 1
    if pipe_check(pipes[pt[0][1]], pipes[pt[0][2]]):
        pipe_connection = True
        pipe_pair = [1,2]
        other = 0
        
    if pipe_connection:
        if elbow_check(pipes[pt[0][pipe_pair[0]]], pipes[pt[0][other]]):
            tee_connections.append((pt, pipe_pair, other))
            
print(len(tee_connections), tee_connections[0])
        

In [ ]:
# check for elbows or connected pipes
pipe_connections = []
elbow_connections = []
for i, ne in enumerate(nearby_edges): 

    if pipe_check(ne[0], ne[1]):
        pipe_connections.append(ne)
        continue
    
    if elbow_check(ne[0], ne[1]):
        elbow_connections.append(ne)
            
        






# remove detected tees

In [ ]:
# check for elbows


In [ ]:
def get_radius_from_bbox(bbox):
    sides = bbox[bbox.argsort()[:2]]
    return (sum(sides)/4)


In [ ]:
# checking and visualisation

elbows = [[n[0], np.array(n[1]), np.array(n[2]), np.array(n[3]), n[4]] for n in nodes if n[0]==1]
tees = [[n[0], np.array(n[1]), np.array(n[2]), np.array(n[3]), n[4]] for n in nodes if n[0]==2]

print(len(tees))

def visualise_tees(tee_connections, blueprint):
    # setup ifc
    ifc = setup_ifc_file(blueprint)
    owner_history = ifc.by_type("IfcOwnerHistory")[0]
    project = ifc.by_type("IfcProject")[0]
    context = ifc.by_type("IfcGeometricRepresentationContext")[0]
    floor = ifc.by_type("IfcBuildingStorey")[0]

    ifc_info = {"owner_history": owner_history,
        "project": project,
       "context": context, 
       "floor": floor}
    
    # calculate tee parameters
    for tee in tee_connections[:1]:
        pipe_pair_ids = [tee[0][0][tee[1][0]], tee[0][0][tee[1][1]]]
        pipe_pair = (pipes[pipe_pair_ids[0]], pipes[pipe_pair_ids[1]])
        other_id = tee[0][0][tee[2]]
        other = pipes[other_id]
        pipe_edge_ids = [tee[0][1][tee[1][0]], tee[0][1][tee[1][1]]]
        other_edge_id = tee[0][1][tee[2]]
        
        r1 = (get_radius_from_bbox(pipe_pair[0][2]) + get_radius_from_bbox(pipe_pair[1][2]))/2
        r2 = get_radius_from_bbox(other[2])
        print('r', r1, r2)     

        l1_edges = [pipe_edges[pipe_pair_ids[0]][pipe_edge_ids[0]], 
                    pipe_edges[pipe_pair_ids[1]][pipe_edge_ids[1]]]
        l1 = np.sqrt(sq_dist_vect(l1_edges[0], l1_edges[1]))
        l2_edges = [(l1_edges[0] + l1_edges[1])/2, pipe_edges[other_id][other_edge_id]]
        l2 = np.sqrt(sq_dist_vect(l2_edges[0], l2_edges[1]))
        print('l', l1, l2)

        p1 = l1_edges[0].tolist()
        p2 = l2_edges[0].tolist()
        d1 = pipe_pair[0][3].tolist()
        d2 = other[3].tolist()
        print('pd', p1, p2, d1, d2)
        
        create_IfcTee(r1, r2, l1, l2, d1, d2, p1, p2, ifc, ifc_info)
        
    return ifc
        

In [ ]:
ifc = visualise_tees(tee_connections, blueprint)

In [ ]:
ifc.write("bp_tee_vis.ifc")